#### The purpose of this script is to run the CRUD operations on the MySQL database for the delfi study. A SQL-heavy implementatation (Python MySQL connector) will be used to access/interact with the database(s).


In [57]:
# import libraries

from dotenv import load_dotenv
import os
import mysql.connector
from mysql.connector import errorcode 
from datetime import date, datetime, timedelta
from decimal import Decimal

### 1) Connecting to the MySQL Server


In [33]:
# Load the credentials from .env file
load_dotenv()

# Access the credentials
config = {
    "host" : os.getenv("DB_HOST"),
    "port" : int(os.getenv("DB_PORT")),
    "user" : os.getenv("DB_USER"),
    "password" : os.getenv("DB_PASSWORD")
}

# Create connection to db 
try:
    db = mysql.connector.connect(**config)
    if db.is_connected():
        print("Connected to MySQL server")

except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your user name or password")
    elif err.erno == errorcode.ER_BAD_DB_ERROR:
        print("Databases do not exist")
    else:
        print(err)



Connected to MySQL server


### 2) Inspect (existing) databases 

#### 2.1) All databases

In [25]:
# Create cursor to enable SQL operations 
cursor = db.cursor(buffered=True) ## A buffered cursor prefetches the entire result set into client memory automatically when you call execute()


# Show existing databases
cursor.execute("SHOW DATABASES")
db_list = cursor.fetchall() # fetching all results to avoid interal read errors due to unfetched results
print("All databases:", db_list)
cursor.close() # close the cursor when done (good practice)

All databases: [('delfi_study',), ('information_schema',), ('lni_study',), ('mysql',), ('performance_schema',), ('sys',)]


True

- cursor = a database object that allows you to iterate over the rows returned by a query

- INFORMATION_SCHEMA provides access to database metadata, information about the MySQL server such as the name of a database or table, the data type of a column, or access privileges
- The MySQL Performance Schema is a feature for monitoring MySQL Server execution at a low level.
    - The Performance Schema provides a way to inspect internal execution of the server at runtime.
    - The Performance Schema monitors server events.


#### 2.2) delfi_study database

In [28]:
# Check tables in db 
cursor = db.cursor(buffered=True)

schema = "delfi_study"
cursor.execute(f"SHOW TABLES FROM {schema}")
tables = cursor.fetchall()  # ensures all rows are read (avoid internal error due to unread result rows)

print(f"\nTables in {schema} database:", tables)
if not tables:
    print(f"No tables found in the {schema} database.")
else:
    print(f"{schema} database has {len(tables)} tables.")

cursor.close() # close the cursor when done (good practice)


Tables in delfi_study database: [('books',), ('papers',)]
delfi_study database has 2 tables.


True

In [29]:
# Get table details (table: books)
cursor = db.cursor(buffered=True)

cursor.execute("DESCRIBE delfi_study.books")
results = cursor.fetchall()
print("\nTable description:")
for row in results:
    print(row)

cursor.close() # close the cursor when done (good practice)


Table description:
('id', 'int', 'NO', 'PRI', None, 'auto_increment')
('title', 'varchar(200)', 'NO', '', None, '')
('author', 'varchar(100)', 'NO', '', None, '')
('pages', 'int', 'NO', '', None, '')
('publication_date', 'date', 'NO', '', None, '')
('price', 'float', 'NO', '', None, '')


True

- cursor.execute() only executes the SQL query—it doesn't display anything. The results are stored in the cursor, and you need to fetch them to see them

In [30]:
# Get table details (table: books)
cursor = db.cursor(buffered=True)

cursor.execute("DESCRIBE delfi_study.papers")
results = cursor.fetchall()
print("\nTable description:")
for row in results:
    print(row)

cursor.close() # close the cursor when done (good practice)


Table description:
('id', 'int', 'NO', 'PRI', None, 'auto_increment')
('authors', 'varchar(500)', 'YES', '', None, '')
('text', 'text', 'YES', '', None, '')


True

#### 2.3) lni_study database

In [31]:
# Check tables in db 
cursor = db.cursor(buffered=True)

schema = "lni_study"
cursor.execute(f"SHOW TABLES FROM {schema}")
tables = cursor.fetchall()  # ensures all rows are read (avoid internal error due to unread result rows)

print(f"\nTables in {schema} database:", tables)
if not tables:
    print(f"No tables found in the {schema} database.")
else:
    print(f"{schema} database has {len(tables)} tables.")

cursor.close() # close the cursor when done (good practice)


Tables in lni_study database: []
No tables found in the lni_study database.


True

### 3) Run CRUD (Create, Read, Update, Delete) operations on the delfi_study database

In [64]:
# Connect to delfi_study db 

# Load the credentials from .env file
load_dotenv()

# Access the credentials
config = {
    "host" : os.getenv("DB_HOST"),
    "port" : int(os.getenv("DB_PORT")),
    "user" : os.getenv("DB_USER"),
    "password" : os.getenv("DB_PASSWORD"), 
    "database": 'delfi_study' #add database parameter
}

# Create connection to db 
try:
    db_delfi = mysql.connector.connect(**config)
    if db_delfi.is_connected():
        print("Connected to delfi db")

except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your user name or password")
    elif err.erno == errorcode.ER_BAD_DB_ERROR:
        print("Databases do not exist")
    else:
        print(err)



Connected to delfi db


#### 3.1) C: Create Tables & Inserting Data 

- https://dev.mysql.com/doc/connector-python/en/connector-python-example-ddl.html
- https://dev.mysql.com/doc/connector-python/en/connector-python-example-cursor-transaction.html


##### 3.1.1) Creating Tables

In [44]:
TABLES = {}

TABLES['employees'] = (
    "CREATE TABLE `employees` (" 
    "  `emp_no` int(11) NOT NULL AUTO_INCREMENT," # backticks = name of column; 11 is the display width of the integer column; preventing null values; automatically generating a unique identifier for each new row 
    "  `birth_date` date NOT NULL,"
    "  `first_name` varchar(14) NOT NULL," # length that indicates the maximum number of characters you want to store
    "  `last_name` varchar(16) NOT NULL,"
    "  `gender` enum('M', 'F') NOT NULL," # An ENUM is a string object with a value chosen from a list of permitted values that are enumerated explicitly in the column specification at table creation time
    "  `hire_date` date NOT NULL,"
    "  PRIMARY KEY (`emp_no`)" 
    ") ENGINE = InnoDB")

TABLES['departments'] = (
    "CREATE TABLE `departments`("
    "  `dept_no` char(4) NOT NULL," # char: The length can be any value from 0 to 255.
    "  `dept_name` varchar(40) NOT NULL,"
    "  PRIMARY KEY (`dept_no`), UNIQUE KEY `dept_name`(`dept_name`)"
    ") ENGINE = InnoDB")

TABLES['salaries'] = (
    "CREATE TABLE `salaries`("
    "  `emp_no` int(11) NOT NULL,"
    "  `salary` int(11) NOT NULL,"
    "  `from_date` date NOT NULL,"
    "  `to_date` date NOT NULL,"
    "  PRIMARY KEY (`emp_no`, `from_date`), KEY `emp_no` (`emp_no`)," # Composite primary keys. Allows multiple salary records per employee, one for each time period
    "  CONSTRAINT `salaries_ibfk_1` FOREIGN KEY (`emp_no`) " # Foreign key: field/column(or collection of fields) in a table that refers to a PRIMARY KEY in another table
    "       REFERENCES `employees` (`emp_no`) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

TABLES['dept_emp'] = (
    "CREATE TABLE `dept_emp` ("
    "  `emp_no` int(11) NOT NULL,"
    "  `dept_no` char(4) NOT NULL,"
    "  `from_date` date NOT NULL,"
    "  `to_date` date NOT NULL,"
    "  PRIMARY KEY (`emp_no`, `dept_no`), KEY `emp_no`(`emp_no`)," # KEY emp_no = create indexes (also called secondary indexes) to speed up queries
    "  KEY `dept_no`(`dept_no`)," # KEY dept_no = create indexes (also called secondary indexes) to speed up queries (could also do that for `from_date`and `to_date``)
    "  CONSTRAINT `dept_emp_ibfk_1` FOREIGN KEY (`emp_no`) "
    "    REFERENCES `employees`(`emp_no`) ON DELETE CASCADE,"
    "  CONSTRAINT `dept_emp_ibfk_2` FOREIGN KEY (`dept_no`) "
    "    REFERENCES `departments` (`dept_no`) ON DELETE CASCADE"
    ") ENGINE = InnoDB")

TABLES['dept_manager'] = (
    "CREATE TABLE `dept_manager` ("
    " `emp_no`int(11) NOT NULL,"
    " `dept_no` char(4) NOT NULL,"
    " `from_date` date NOT NULL,"
    "  `to_date`date NOT NULL,"
    "  PRIMARY KEY (`emp_no`, `dept_no`),"
    "  KEY `emp_no` (`emp_no`),"
    "  KEY `dept_no`(`dept_no`),"
    "  CONSTRAINT `dept_manager_ibfk_1` FOREIGN KEY (`emp_no`) "
    "    REFERENCES `employees`(`emp_no`) ON DELETE CASCADE,"
    "  CONSTRAINT `dept_manager_ibfk_2` FOREIGN KEY (`dept_no`) "
    "     REFERENCES `departments`(`dept_no`) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

TABLES['titles'] = (
    "CREATE TABLE `titles`("
    "  `emp_no`int(11) NOT NULL,"
    "  `title`varchar(50) NOT NULL,"
    "  `from_date`date NOT NULL,"
    "  `to_date`date DEFAULT NULL,"
    "  PRIMARY KEY (`emp_no`, `title`, `from_date`), KEY `emp_no` (`emp_no`),"
    "  CONSTRAINT `titles_ibfk_1` FOREIGN KEY (`emp_no`)"
    "    REFERENCES `employees` (`emp_no`) ON DELETE CASCADE"
    ") ENGINE=InnoDB")


- InnoDB is a general-purpose storage engine that balances high reliability and high performance. In MySQL 8.4, InnoDB is the default MySQL storage engine. Unless you have configured a different default storage engine, issuing a CREATE TABLE statement without an ENGINE clause creates an InnoDB table.([Documentation](https://dev.mysql.com/doc/refman/8.4/en/innodb-introduction.html))

- A MySQL Primary Key is a unique column/field in a table that should not contain duplicate or NULL values and is used to identify each record in the table uniquely. The role of the primary key constraint is to maintain the integrity of the database by preventing duplicate and null values in the key column. We can use the AUTO_INCREMENT attribute on the primary key field so that a new value is automatically added to the column when we create a new row. A primary key makes the query execution fast, as MySQL automatically creates a "Primary" index for the primary key. ([Source](https://www.geeksforgeeks.org/mysql/mysql-primary-key/))

- A MySQL unique key is a constraint that ensures all values in a column or a set of columns are distinct, preventing duplicate entries. Unlike a primary key, a unique key can allow null values and multiple unique keys can exist in a table. A UNIQUE constraint in MySQL ensures that all values in a column or a set of columns are distinct from one another. This constraint is used to prevent duplicate entries in a column or combination of columns, maintaining data integrity. ([Source](https://www.geeksforgeeks.org/mysql/mysql-unique-constraint/))

- Composite primary keys: enforce uniqueness on the combination of values, not individually. No single column uniquely identifies a row (like the salary example above)

- FOREIGN KEY (emp_no) REFERENCES employees (emp_no):
    - Every emp_no in salaries must exist in employees
    - You can't insert a salary for non-existent employee

- A FOREIGN KEY is a field/column(or collection of fields) in a table that refers to a PRIMARY KEY in another table. It is used for linking one or more than one table together. FOREIGN KEY is also called referencing key. A Foreign key creates a link (relation) between two tables thus creating referential integrity. ([Source](https://www.geeksforgeeks.org/mysql/mysql-foreign-key-constraint/))

- ON DELETE CASCADE: if an employee is deleeted, their salary records are automatically deleted too. MySQL provides a more effective way called ON DELETE CASCADE referential action for a foreign key that allows you to delete data from child tables automatically when you delete the data from the parent table. ([Source](https://www.mysqltutorial.org/mysql-basics/mysql-on-delete-cascade/)) 

- Identation:  doesn't matter in SQL syntax. MySQL ignores whitespace (spaces, tabs, newlines). Is only for human readability.



In [45]:
# Create cursor
cursor = db_delfi.cursor(buffered=True)

# Creating tables
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end="")
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists")
        else:
            print(err.msg)
    else:
        print("OK")

# Close cursor
cursor.close()

Creating table employees: OK
Creating table departments: OK
Creating table salaries: OK
Creating table dept_emp: OK
Creating table dept_manager: OK
Creating table titles: OK


True

- In a real application, we would typically avoid the error condition entirely by using the IF NOT EXISTS clause of the [CREATE TABLE](https://dev.mysql.com/doc/refman/8.0/en/create-table.html) statement.

In [46]:
# Check tables in db 
cursor = db_delfi.cursor(buffered=True)

cursor.execute(f"SHOW TABLES")
tables = cursor.fetchall()  # ensures all rows are read (avoid internal error due to unread result rows)

print(f"\nTables in delfi database:", tables)

# Close cursor
cursor.close()


Tables in delfi database: [('books',), ('departments',), ('dept_emp',), ('dept_manager',), ('employees',), ('papers',), ('salaries',), ('titles',)]


True

##### 3.1.2) Inserting Data

In [48]:
cursor = db_delfi.cursor(buffered=True)

tomorrow = datetime.now().date() + timedelta(days = 1)

add_employee = ("INSERT INTO employees "
                "(first_name, last_name, hire_date, gender, birth_date) "
                "VALUES (%s, %s, %s, %s, %s)") #%s is a SQL parameter placeholder

add_salary = ("INSERT INTO salaries "
              "(emp_no, salary, from_date, to_date) "
              "VALUES (%(emp_no)s, %(salary)s, %(from_date)s, %(to_date)s)")

data_employee = ('Geert', 'Vanderkelen', tomorrow, 'M', date(1977, 6, 14)) #tuple

# Insert new employee
cursor.execute(add_employee, data_employee)
emp_no = cursor.lastrowid


data_salary = { #dict
    'emp_no': emp_no, 
    'salary': 50000,
    'from_date' : tomorrow,
    'to_date': date(9999, 1, 1)
}

# Insert salary information
cursor.execute(add_salary, data_salary)

# Make sure data is committed to the database
db_delfi.commit() 

# Close cursor
cursor.close()

True

- Relevance of placeholders in SQL Insert:
    - SQL Injection Prevention — Without placeholders, malicious input can execute arbitrary SQL
    - Proper data type handling — MySQL connector correctly formats dates, strings, None/NULL, etc.
    - Character escaping — Handles quotes, backslashes, special characters automatically

- Commit: Since by default Connector/Python turns autocommit off, and MySQL 5.5 and higher uses transactional InnoDB tables by default, it is necessary to commit your changes using the connection's commit() method

#### 3.2) R: Reading/selecting/querying data 

- https://dev.mysql.com/doc/connector-python/en/connector-python-example-cursor-select.html

The task is to select all employees hired in the year 2025 and print their names and hire dates to the console.

In [56]:
cursor = db_delfi.cursor()

query = ("SELECT first_name, last_name, hire_date FROM employees "
         "WHERE hire_date BETWEEN %s AND %s")

hire_start = date(2025, 1, 1)
hire_end = date(2025, 12, 31)

cursor.execute(query, (hire_start, hire_end))

for (first_name, last_name, hire_date) in cursor:
    print("{}, {} was hired on {:%d %b %Y}".format(
        last_name, first_name, hire_date
    ))

cursor.close()

Vanderkelen, Geert was hired on 16 Dec 2025


True

- Connector/Python converts hire_start and hire_end from Python types to a data type that MySQL understands and adds the required quotes. In this case, it replaces the first %s with '1999-01-01', and the second with '1999-12-31'.

- The data used to replace the %s-markers in the query is passed as a tuple: (hire_start, hire_end).

- After executing the query, the MySQL server is ready to send the data. The result set could be zero rows, one row, or 100 million rows. Depending on the expected volume, you can use different techniques to process this result set. In this example, we use the cursor object as an iterator. The first column in the row is stored in the variable first_name, the second in last_name, and the third in hire_date.

#### 3.3) U: Update data 

- https://dev.mysql.com/doc/connector-python/en/connector-python-tutorial-cursorbuffered.html

The following example script gives a long-overdue 15% raise effective tomorrow to all employees who joined in the year 2025 and are still with the company.

In [66]:
# Get two buffered cursors
curA = db_delfi.cursor(buffered=True)
curB = db_delfi.cursor(buffered=True)

# Query to get employees who joined in a period defined by two dates
query = (
    "SELECT s.emp_no, salary, from_date, to_date FROM employees AS e " # 'e' is alias for employees table
    "LEFT JOIN salaries AS s USING (emp_no) " # 's' is alias for salaries table
    "WHERE to_date = DATE('9999-01-01')"
    "AND e.hire_date BETWEEN DATE(%s) AND DATE(%s)")

# UPDATE and INSERT statements for the old and new salary 
update_old_salary = (
    "UPDATE salaries SET to_date = %s "
    "WHERE emp_no = %s AND from_date = %s"
)
insert_new_salary = (
    "INSERT INTO salaries (emp_no, from_date, to_date, salary) "
    "VALUES (%s, %s, %s, %s) "
    "ON DUPLICATE KEY UPDATE salary = VALUES(salary), to_date = VALUES(to_date)"
)

# Select the employees getting a raise
curA.execute(query, (date(2025, 1, 1), date(2025, 12, 31)))

# Iterate through the result of curA
for (emp_no, salary, from_date, to_date) in curA:

    # Update the old and insert the new salary
    new_salary = int(round(salary * Decimal('1.15')))
    curB.execute(update_old_salary, (tomorrow, emp_no, from_date))
    curB.execute(insert_new_salary,
                 (emp_no, tomorrow, date(9999, 1, 1), new_salary))
    
    # Commit the changes
    db_delfi.commit()

# Close the cursors
curA.close()
curB.close()

True

- Buffered cursor: To iterate through the selected employees, we use buffered cursors. (A buffered cursor fetches and buffers the rows of a result set after executing a query)

- LEFT JOIN: 
    - The LEFT JOIN clause selects data starting from the left table (employees), matching each row from the left table (employees) with every corresponding row from the right table(salaries) based on the join_condition. [Source](https://www.mysqltutorial.org/mysql-basics/mysql-left-join/)
    - MySQL JOIN statements enable merging tables based on common columns.
    - MySQL LEFT JOIN is a type of outer join that **returns all records from the left table and matches records from the right table.** If there is no match, the result is NULL from the right table. This join is also known as Left Outer Join. [Source](https://www.geeksforgeeks.org/mysql/mysql-left-join/)

In [67]:
# Check that the salary update was successfully 

cursor = db_delfi.cursor()

# Query to see all salary records for employees hired in 2025
query = (
    "SELECT e.emp_no, e.first_name, e.last_name, s.salary, s.from_date, s.to_date "
    "FROM employees AS e "
    "JOIN salaries AS s ON e.emp_no = s.emp_no "
    "WHERE e.hire_date BETWEEN %s AND %s "
    "ORDER BY e.emp_no, s.from_date"
)

cursor.execute(query, (date(2025, 1, 1), date(2025, 12, 31)))

print("\nSalary records for employees hired in 2025:")
for (emp_no, first_name, last_name, salary, from_date, to_date) in cursor:
    print(f"Employee {emp_no} ({first_name} {last_name}): "
          f"${salary} from {from_date} to {to_date}")
    
cursor.close()


Salary records for employees hired in 2025:
Employee 1 (Geert Vanderkelen): $57500 from 2025-12-16 to 9999-01-01


True

#### 3.4) D: Delete data 

- https://www.geeksforgeeks.org/python/crud-operation-in-python-using-mysql/

In [68]:
# 1) Show all tables in delfi_db again
cursor = db_delfi.cursor(buffered=True)

cursor.execute(f"SHOW TABLES")
tables = cursor.fetchall()  # ensures all rows are read (avoid internal error due to unread result rows)

print(f"\nTables in delfi database:", tables)

# Close cursor
cursor.close()


Tables in delfi database: [('books',), ('departments',), ('dept_emp',), ('dept_manager',), ('employees',), ('papers',), ('salaries',), ('titles',)]


True

In [70]:
# 2) Delete a single row from a table 

cursor = db_delfi.cursor()

query_delete_entry = "DELETE FROM employees WHERE emp_no=1"

cursor.execute(query_delete_entry)

db_delfi.commit()

cursor.close()

True

In [71]:
# 3) Check that the delete was successfully 

cursor = db_delfi.cursor()

# Query to see all salary records for employees hired in 2025
query = (
    "SELECT e.emp_no, e.first_name, e.last_name, s.salary, s.from_date, s.to_date "
    "FROM employees AS e "
    "JOIN salaries AS s ON e.emp_no = s.emp_no "
    "WHERE e.hire_date BETWEEN %s AND %s "
    "ORDER BY e.emp_no, s.from_date"
)

cursor.execute(query, (date(2025, 1, 1), date(2025, 12, 31)))

print("\nSalary records for employees hired in 2025:")
for (emp_no, first_name, last_name, salary, from_date, to_date) in cursor:
    print(f"Employee {emp_no} ({first_name} {last_name}): "
          f"${salary} from {from_date} to {to_date}")
    
cursor.close()


Salary records for employees hired in 2025:


True

In [74]:
# 4) Delete entire tables (which have no foreign keys)

cursor = db_delfi.cursor()

cursor.execute(f"DROP TABLE IF EXISTS books")

cursor.close()

True

In [75]:
# 5) Check that deletion was successful
cursor = db_delfi.cursor(buffered=True)

cursor.execute(f"SHOW TABLES")
tables = cursor.fetchall()  # ensures all rows are read (avoid internal error due to unread result rows)

print(f"\nTables in delfi database:", tables)

# Close cursor
cursor.close()


Tables in delfi database: [('departments',), ('dept_emp',), ('dept_manager',), ('employees',), ('papers',), ('salaries',), ('titles',)]


True

In [76]:
# 6) Delete entire tables (which have foreign keys) -> order of deletion is relevant!

cursor = db_delfi.cursor()

# Drop in order: tables with foreign keys first, referenced tables last

tables_to_drop = [
    'titles',
    'salaries',
    'dept_manager',
    'dept_emp',
    'departments',
    'employees'
]

for table in tables_to_drop:
    cursor.execute(f"DROP TABLE IF EXISTS {table}")
    print(f"Dropped {table}")

db_delfi.commit()
cursor.close()

Dropped titles
Dropped salaries
Dropped dept_manager
Dropped dept_emp
Dropped departments
Dropped employees


True

In [77]:
# 7) Check that deletion was successful
cursor = db_delfi.cursor(buffered=True)

cursor.execute(f"SHOW TABLES")
tables = cursor.fetchall()  # ensures all rows are read (avoid internal error due to unread result rows)

print(f"\nTables in delfi database:", tables)

# Close cursor
cursor.close()


Tables in delfi database: [('papers',)]


True

In [78]:
# 8) Delete the last table (papers). Will create own new table when papers are pre-processed and inserted to MySQL db

cursor = db_delfi.cursor()

cursor.execute(f"DROP TABLE IF EXISTS papers")

cursor.close()

True

In [79]:
# 9) Check that deletion was successful
cursor = db_delfi.cursor(buffered=True)

cursor.execute(f"SHOW TABLES")
tables = cursor.fetchall()  # ensures all rows are read (avoid internal error due to unread result rows)

print(f"\nTables in delfi database:", tables)

# Close cursor
cursor.close()


Tables in delfi database: []


True

In [80]:
db_delfi.close()